In [1]:
import numpy as np 
import pygame 
import math
import sys 
from copy import deepcopy 
import time 
import random 
import os
import signal
import tensorflow as tf

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from Ataxx.Ataxx.AtaxxGame import AtaxxGame
import Ataxx.Ataxx.AtaxxPlayers as APlayers

from Go.Go.GoGame import GoGame
import Go.Go.GoPlayers as GPlayers

In [5]:
"""
Para escolher o jogo basta alterar a variável Game
Caso se deseje jogar o ataxx com o tamanho do tabuleiro variável(flex) basta escrever AFx e o tamanho é escolhido aleatoriamente
"""
Game="A6x6" # "A4x4" "AFx" "A6x6" "G7x7" "G9x9" 

if "Fx" in Game:
    SIZE = random.choice([4, 5, 6]) # Para escolher manualmente o tamanho basta trocar random.choice([4, 5, 6]) pelo número desejado
    Game += str(SIZE)
else:
    SIZE = int(Game[-1])

print(f"Game -> {Game}")

SQUARE_SIZE = 75 #Tamanho dos quadrados da interface


#Cores
BLUE = (0,0,255) 
RED = (255,0,0) 
WHITE_COLOR = (255,255,255) 
BLACK_COLOR = (0,0,0) 
BACKGROUND_COLOR = (240, 196, 52)

#Tamanhos da interface
width = SIZE * SQUARE_SIZE 
height = SIZE * SQUARE_SIZE 
width = SIZE * SQUARE_SIZE 
height = SIZE * SQUARE_SIZE 
screen_size = (width, height) 


Game -> G9x9


In [6]:
import socket
import time

def receive_data_with_timeout(agent, timeout):
    """
    Função para receber a mensagem do agente com um limite de tempo
    """
    agent.settimeout(timeout)
    start_time = time.time()
    received_data = b""
    while True:
        try:
            data = agent.recv(1024)
            if data:
                received_data += data
                break
            print(received_data)
            if time.time() - start_time > timeout:
                break
        except socket.timeout:
            # Tempo limite excedido
            return False
    return received_data.decode()






def start_server(host='localhost', port=12345):
    """
    Função para iniciar o servidor
    """
    pygame.init()
    
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(2)

    print("Waiting for two agents to connect...")
    agent1, addr1 = server_socket.accept()
    print("Agent 1 connected from", addr1)
    bs=b'AG1 '+Game.encode()
    agent1.sendall(bs)

    agent2, addr2 = server_socket.accept()
    print("Agent 2 connected from", addr2)
    bs=b'AG2 '+Game.encode()
    agent2.sendall(bs)    

    agents = [agent1, agent2]
    current_agent = 0
    curPlayer = 1
    invalid_counter = 0
    
    jog=0
    
    screen = pygame.display.set_mode(screen_size)  
    if Game[0] == 'G':
        screen.fill(BACKGROUND_COLOR)
    pygame.display.set_caption('Server')
    board = game.getInitBoard()
    game.display(board, screen)

    
    time.sleep(10) # Após os dois agentes se conectarem faz uma pausa de 10 segundos antes de começar o jogo

    while True:
        pygame.event.pump()

        try:
            if Game[0] == "A": # Se o jogo é o ataxx
                if invalid_counter >= 3: #Se o agente fizer 3 jogadas inválidas seguidas envia mensagem ao agente que perdeu a vez e ao outro agente a avisar que é a sua vez
                    agents[current_agent].sendall(b'NOT YOU')
                    agents[1-current_agent].sendall(b'YOU')
                    current_agent = 1-current_agent
                    curPlayer = -curPlayer
                    invalid_counter = 0
                    continue

                if game.getGameEnded(board, curPlayer) is not None: # Se o jogo acabou

                    player1_counter, player2_counter = game.piece_counter(board)
                    winner = curPlayer * game.getGameEnded(board, curPlayer)

                    message = f"END {winner} {player1_counter} {player2_counter}"
                    print(message)

                    agents[current_agent].sendall(message.encode())
                    agents[1-current_agent].sendall(message.encode())
                    break

                data = receive_data_with_timeout(agents[current_agent], 10)
                if not data: # Se não recebeu a mensagem com o movimento
                    print(f"Agent {current_agent} time limit")

                    agents[current_agent].sendall(b'NOT YOU')
                    agents[1-current_agent].sendall(b'YOU')

                    current_agent = 1-current_agent
                    curPlayer = -curPlayer
                    invalid_counter = 0
                    continue

                # Process the move (example: "MOVE X,Y")
                print(current_agent, " -> ",data)
                if is_valid_move(board, curPlayer, data):
                    agents[current_agent].sendall(b'VALID')
                    agents[1-current_agent].sendall(data.encode())

                    move = (int(data[5]), int(data[7]), int(data[9]), int(data[11]))
                    board, curPlayer = game.getNextState(board, curPlayer, move)
                    game.display(board, screen)
                    current_agent = 1-current_agent
                    invalid_counter = 0
                    time.sleep(3)
                else:
                    agents[current_agent].sendall(b'INVALID')
                    invalid_counter += 1
        
        
        
            elif Game[0] == "G": # Se o jogo é o go (processo semelhante ao do ataxx)
                if invalid_counter >= 3:
                    agents[current_agent].sendall(b'NOT YOU')
                    agents[1-current_agent].sendall(b'YOU')
                    current_agent = 1-current_agent
                    curPlayer = -curPlayer
                    invalid_counter = 0
                    continue
    
    
                if game.getGameEnded(board, curPlayer) is not None: 
                    winner, player1_score, player2_score = game.getGameEnded(board, curPlayer, game_over=True, get_score=True)

                    winner = curPlayer * winner
                    message = f"END {winner} {player1_score} {player2_score}"
                    print(message)
                    agents[current_agent].sendall(message.encode())
                    agents[1-current_agent].sendall(message.encode())

                    break
    
                data = receive_data_with_timeout(agents[current_agent], 10)
                if not data:
                    print(f"Agent {current_agent} time limit")
                    agents[current_agent].sendall(b'NOT YOU')
                    agents[1-current_agent].sendall(b'YOU')
                    current_agent = 1-current_agent
                    curPlayer = -curPlayer
                    invalid_counter = 0
                    continue
                
                print(current_agent, " -> ",data)
                if is_valid_move(board, curPlayer, data):
                    agents[current_agent].sendall(b'VALID')
                    agents[1-current_agent].sendall(data.encode())

                    move = (int(data[5]), int(data[7]))
                    board, curPlayer = game.getNextState(board, curPlayer, move)
                    game.display(board, screen)
                    current_agent = 1-current_agent
                    invalid_counter = 0
                    time.sleep(3)
                else:
                    agents[current_agent].sendall(b'INVALID')
                    invalid_counter += 1
            
        except Exception as e:
            print("Error:", e)
            break
        

    print("\n-----------------\nGAME END\n-----------------\n")
    time.sleep(1)
    pygame.init()
    game_over = True
    while game_over: # A interface continua aberta até ser fechada clicando no botão para fechar
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT:

                agent1.close()
                agent2.close()
                server_socket.close()
                pygame.quit()
                game_over = False

def is_valid_move(board, player, move):
    """
    Função para verificar se o movimento é válido
    """
    if Game[0] == 'A':
        x1, y1, x2, y2 = int(move[5]), int(move[7]), int(move[9]), int(move[11]) 
        legal_moves = game.get_legal_moves(board, player)
        if ((x1, y1), (x2, y2)) in legal_moves:
            return True
        return False


    
    elif Game[0] == 'G':
        
        valids = game.getValidMoves(game.getCanonicalForm(board, player), 1)
        x, y = int(move[5]), int(move[7])
        if x==9 and y==9: # Passar a jogada
            a = game.size*game.size
        else:
            a = game.size * x + y

        return valids[a] == 1

        
if __name__ == "__main__":
    if Game[0] == 'A':
        game = AtaxxGame(SIZE)
    elif Game[0] == 'G':
        game = GoGame(SIZE)
    start_server()
    


Waiting for two agents to connect...
Agent 1 connected from ('127.0.0.1', 50777)
Agent 2 connected from ('127.0.0.1', 50780)
0  ->  MOVE 0,1
1  ->  MOVE 0,4
0  ->  MOVE 5,5
1  ->  MOVE 6,2
0  ->  MOVE 7,7
1  ->  MOVE 6,3
0  ->  MOVE 1,2
1  ->  MOVE 8,3
0  ->  MOVE 9,9
1  ->  MOVE 9,9
END -1 5 9.5

-----------------
GAME END
-----------------

